In [27]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

In [28]:
NEIGHBORS_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/'

In [29]:
POLISHED_DIRECT_PATH = '/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/'

In [30]:
DIRECT_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/DIRECT fits/'  # THIS IS SPECIFIC TO MY DRIVE, YOU WILL HAVE TO CREATE YOUR OWN FOLDER AND THUS CHANGE THE PATH NAME

In [31]:
DIFEV_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits/'

In [32]:
BASIN_HOPPING_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/BASIN_HOPPING_FITS/'

In [33]:
SHGO_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/SHGO_FITS/'

In [34]:
EXHUASTIVE_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Exhuastive fits/'

In [35]:
BOBBY_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/BOBBY FITS/'

In [36]:
POLISH0_PATH='/'

In [37]:
import pickle
from datetime import date
def save_fit(fit, participant,algo):
  tdy=date.today()
  sp=""
  if algo=="direct":
    sp=DIRECT_SAVE_PATH
  elif algo=="diffev":
    sp=DIFEV_SAVE_PATH
  elif algo=="basin_hopping":
    sp=BASIN_HOPPING_SAVE_PATH
  elif algo=="shgo":
    sp=SHGO_SAVE_PATH
  elif algo=="brute":
    sp=EXHUASTIVE_SAVE_PATH
  elif algo=='bobby':
    sp=BOBBY_PATH
  elif algo=='neighbors':
    sp=NEIGHBORS_PATH
  elif algo=='polished_direct':
    sp=POLISHED_DIRECT_PATH



  fname=sp+"participant " + str(participant)+" "+tdy.strftime("%B %d, %Y")+".pickle"
  pickle_out=open(fname,"wb")
  pickle.dump(fit,pickle_out)
  pickle_out.close()



In [38]:
def load_fit(path):
  pickle_in=open(path,"rb") # put the path here
  fit=pickle.load(pickle_in)
  return fit


In [39]:
p = {
    1: 0.03,
    2: 0.06,
    3: 0.09,
    4: 0.12,
    5 : 0.14,
    6: 0.11,
    7: 0.09,
    8: 0.08,
    9: 0.07,
    10: 0.06,
    11: 0.05,
    12: 0.04,
    13: 0.03,
    14: 0.02,
    15: 0.01
}
I=15

In [40]:
all_data= pd.read_csv('data/All Data EUT 230519.csv')

In [41]:

SellData= all_data.copy()

In [42]:
columns=list(SellData.columns)

In [43]:
for string in columns:
  if "Predicted"  in string:
      columns.remove(string)
print(columns)

['Subject', 'Day', 'Stored68', 'Price68', 'Sold68', 'Stored67', 'Price67', 'Sold67', 'Stored66', 'Price66', 'Sold66', 'Stored65', 'Price65', 'Sold65', 'Stored64', 'Price64', 'Sold64', 'Stored63', 'Price63', 'Sold63', 'Stored62', 'Price62', 'Sold62', 'Stored61', 'Price61', 'Sold61', 'Stored60', 'Price60', 'Sold60', 'Stored59', 'Price59', 'Sold59', 'Stored58', 'Price58', 'Sold58', 'Stored57', 'Price57', 'Sold57', 'Stored56', 'Price56', 'Sold56', 'Stored55', 'Price55', 'Sold55', 'Stored54', 'Price54', 'Sold54', 'Stored53', 'Price53', 'Sold53', 'Stored52', 'Price52', 'Sold52', 'Stored51', 'Price51', 'Sold51', 'Stored50', 'Price50', 'Sold50', 'Stored49', 'Price49', 'Sold49', 'Stored48', 'Price48', 'Sold48', 'Stored47', 'Price47', 'Sold47', 'Stored46', 'Price46', 'Sold46', 'Stored45', 'Price45', 'Sold45', 'Stored44', 'Price44', 'Sold44', 'Stored43', 'Price43', 'Sold43', 'Stored42', 'Price42', 'Sold42', 'Stored41', 'Price41', 'Sold41', 'Stored40', 'Price40', 'Sold40', 'Stored39', 'Price39', '

In [44]:
error_cols=columns[-10:]

In [45]:
columns.remove( 'OEHold')
columns.remove( 'OHold')

columns.remove('OESold')
columns.remove('OSold')
columns.remove( 'MSold')

columns.remove( 'Diff')
columns.remove( 'Tot')
columns.remove( 'MDev')
columns.remove( 'MDev0')
columns.remove('Tot0')



In [46]:
SellData=all_data[columns]
error_data=all_data[error_cols]

In [47]:
df_days=[]
df_start=SellData.iloc[:,[0]]
splitting_df= SellData.drop("Subject", axis=1)


In [48]:
def split(df):
  cols=list(df.columns)
  stored_cols=[x for x in cols if ("Stored" in x)]
 #for i in DAYS:
  #cols_for_day=[]

  for name in stored_cols:
    # print("Stored+ day: ", name)

    stored_i=cols.index(name)
    # print("stored index",stored_i)

    new_df= df.iloc[:, list(range(stored_i,stored_i+3))]
    # print("length of new df : ", len(new_df.columns))
    # print("columns of new df : ", list(new_df.columns))
    new_df= pd.concat([df_start,new_df], axis=1)
    df_days.insert(stored_i, new_df)
  # print("length of df_days: ", len(df_days))




In [49]:
split(splitting_df)

In [50]:
df_days.reverse()

In [51]:
DAYS=list(range(0,68))

In [52]:
import math

def prelec(p, gamma):
    x=math.exp(-(-math.log(p)) ** gamma)

    return x

In [53]:

def h_probPTv2(day, f, c_o,gam):
  prob=1.0
  for h in range(1,f+1):
    prob*=sum(prelec(p[j],gam) for j in range(1,c_o[day-h]))
  return prob

In [54]:
def obj_gain(day, price, sold, alpha, beta, lam, gam, c_o,j):
  gain=sold*(price-j)
  gain=gain**alpha
  gain*=prelec(p[j],gam)
  return gain


In [55]:
def obj_loss(day, price, sold, alpha, beta, lam, gam, c_o,j):
  loss=sold*(j-price)
  loss=loss**beta
  loss*=lam*prelec(p[j],gam)
  return loss

In [56]:

def PTv3(day,price,sold,alpha,beta,lam,gam,c_o):
  util=0
  for j in range(c_o[day-1],price):
    util+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for j in range(max(price+1,c_o[day-1]), I+1):

    util-=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for k in range(1,day+2):

    h_prob=h_probPTv2(day,k-1,c_o,gam)
    #h_prob=h_probPT(day_f=k, actual_day=day,gamma=gam,cutoff=c_o)

    gain_prob=0
    for j in range(c_o[day-k],price):
      gain_prob+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)

    loss_prob=0
    for j in range(max(price+1,c_o[day-k]), I+1):
      loss_prob+=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)

    util+=h_prob*(gain_prob-loss_prob)




  #print("util for day",day,"is ", util)
  return util





In [57]:
def max_units(day,price, stored,a,b,l,g,cut_offs):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PTv3(day,price,units,a,b,l,g,cut_offs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [ ]:
def info_return(day, participant):
  curr_df= df_days[day].iloc[participant] # data acquiring is fine.
  n_1=curr_df["Sold"+str(day+1)]
  #print("Sold units on day :",day, " is : ", n_1)

  N_d=curr_df["Stored"+str(day+1)]
  #print("Stored on day",day,"is:", N_d)
  i_d=curr_df["Price"+str(day+1)]
 # print("Price on day",day,"is",i_d)
  return(n_1,N_d,i_d)

In [ ]:
def cutoff_pt_list(a,b,l,g):#this function generates a list before we make our predicitons
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0:
      continue
    price=cutoffs[day-1]
    prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)
    counts=0
    while (prosp <= 0 and price < 15):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                prosp = PTv3(day,price,1,a,b,l,g,cutoffs)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
   # print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs





In [ ]:
def fitPTv2(participant,a,b,l,g):
  predictions=[]                 #[0]*68
  #cut_offs=c_o
  cut_offs=cutoff_pt_list(a,b,l,g)            #cutoff_PT(a,b,l,g,participant)


  for day in DAYS: # 1,2,3
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]

    ''' c_o=cutoff_PT(a,b,l,g,day,cut_offs,stored)
    print("cut off price for day", day,"is ", c_o)
    cut_offs[day-1]=c_o'''
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units(day,price,stored,a,b,l,g, cut_offs)
      else:
        print(".")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  print("DATA ADDED :)")
  return predictions,cut_offs

In [ ]:
def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  t=int(tw)

  if t>day: # 68,67,66
    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)
    print("util PT : ", util)
  else:
    gain=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(c_o[t-1],price))
    loss=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(max(c_o[t-1],price+1),I+1))
    util=gain-loss
    print("util day:" ,day,util)
  return util

In [ ]:
def cutoff_pt_tw_list(a,b,l,g,tw):
  t=tw
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]
    if day<t:
          prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    else:
          prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                if day<t:
                  prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
                else:
                  prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [ ]:
def max_units_PTTW(day,price,stored,a,b,l,g,tw,cutoffs ):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PT_TW(day,price,units,a,b,l,g,tw,cutoffs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [ ]:
def fitPT_TW(participant,a,b,l,g,tw):
  predictions=[]                 #[0]*68
  cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)
  #cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)            #cutoff_PT(a,b,l,g,participant)

  rev_d=reversed(DAYS)
  #rev_d=DAYS
  for day in rev_d: # 67,66,65...
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units_PTTW(day,price,stored,a,b,l,g,tw,cut_offs)
      # else:
        # print("cutoff value of", cut_offs[day], "bigger than price", price, "HOLD PREDICTED ")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  # print("DATA ADDED :)")
  return predictions,cut_offs

In [ ]:
def h_probPT_DD(day, f, c_o,gam):
  prob=1.0
  for h in range(day-1+1,f+2): # day is added 1  here
    prob*=sum(prelec(p[j],gam) for j in range(1,c_o[day-1])) # I don't think this will work, replacement is c_o[day-h] as that is what Yudelson did TALK tO GLASS # Trying c_o[day-1]
  return prob

In [ ]:
def PT_DD(day,price,sold,alpha,beta,lam,gam,c_o, k):
  # since day starts from 0, 1, 2,3,4 for the list DAYS but the equation takes d=1 as the last day so all days in this equation are added 1
  lb_1=c_o[day-1] # this stays the same as it is just the last days cutoff value
  ub_1=price
  gain= sum(
      (
          prelec(p[j],gam)*(sold*(price-j)/(1+k*(day-k+1)))**alpha # day is added 1 here

       )
      for j in range(lb_1, ub_1)
      )
  lb_2=max(lb_1,price+1)
  ub_2=I+1
  loss= sum(

            (
              prelec(
                  (p[j] + sum(h_probPT_DD(day,f,c_o,gam) for f in range(day-1,0,-1))*p[j] # day is added 1 here edit. not anymore
               ),gam)


            )
            * lam*(sold*(j-price)/(1+k*(day-k+1)))**beta # day is added 1 here
            for j in range(lb_2,ub_2)

  )

  util=gain-loss
  print(gain)
  print(loss)
  return util


In [ ]:
def cutoff_PT_DD_list(a,b,l,g,k):

  cutoffs=[1] #last day (d=1) but we used day=0
  for day in DAYS:#0,1,2,3 but actual days start 1, 2,3,4
    if day==0: # last day
      continue
    price=cutoffs[day-1] # last day, also the same as the lower bound for the gain in PT-DD


    prosp = PT_DD(day,price,1,a,b,l,g,cutoffs,k) # sold is one as long as 1 will sell its fine
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1

                prosp = PT_DD(day,price,1,a,b,l,g,cutoffs,k)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [ ]:
#THANK YOU ORION
soldOnly = SellData
for name in SellData.columns :
  if 'Sold' not in name:
    soldOnly=soldOnly.drop(name, axis=1)

soldOnly = soldOnly.T
soldOnly = soldOnly.iloc[::-1]

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
Sold1,4,3,3,4,4,4,4,4,4,4,...,3,4,3,10,6,3,2,12,2,2
Sold2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
Sold3,0,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,1,1,0,2,1
Sold4,28,26,12,12,12,12,7,12,6,12,...,5,4,3,20,6,2,1,5,3,1
Sold5,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sold64,0,0,0,0,0,0,0,0,0,0,...,0,1,1,4,1,1,0,1,0,2
Sold65,0,5,2,5,5,7,3,7,0,7,...,3,3,2,4,1,7,5,0,3,4
Sold66,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,1
Sold67,0,0,0,0,0,0,0,0,0,0,...,0,1,3,0,1,0,2,1,1,0


# ERRORS:


In [ ]:
from scipy.optimize import minimize
from sklearn.metrics import d2_absolute_error_score as d2a
from sklearn.metrics import mean_squared_error

In [ ]:
def avg_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)
    stored=vals[1]
    sold=vals[0]
    if stored==0:
      d0+=1
      continue
    else:
      pred=preds[day]
      #pred=predictionsOnly._get_value(day,"Participant"+str(participant))
      day_err=(pred-sold)/stored
      total+=day_err
  return total/(68.0-d0)


In [ ]:
def get_loss_one(params,participant):#a,b,l,g
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return mse


In [ ]:
def d2a_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return 1-error


In [ ]:
def fit_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]



  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  actuals=actuals[::-1]

  error=d2a(actuals,preds)

  return 1-error

In [ ]:
def fit_errorPTTW_dir(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)


  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)

  error=d2a(actuals,preds)

  return error

In [ ]:
def count_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]

  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[day]
    day_err=abs(sold-pred)
    total+=day_err
  return total

In [ ]:
def count_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)

  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS[::-1]:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[67-day]
    day_err=abs(sold-pred)
    total+=day_err
  return total


In [ ]:
def display_errors(four_fits, start,stop):
  avg_errors=[]
  d2a_errors=[]
  mse_errors=[]
  count_errors=[]
  iter=list(zip(four_fits, list(range(start,stop))))
  for fit,participant in iter:
    x=avg_error(fit,participant)
    y=d2a_error(fit,participant)
    z=get_loss_one(fit,participant)
    m=count_error(fit,participant)
    avg_errors.append(x)
    d2a_errors.append(y)
    mse_errors.append(z)
    count_errors.append(m)
  return avg_errors,d2a_errors,mse_errors,count_errors

In [ ]:
losstest=fit_errorPT_TW(np.array([1,1,1,1,68]),0)

-11.22
prosp on day 1 is  -9.219999999999999
prosp on day 1 is  -7.22
prosp on day 1 is  -5.220000000000001
prosp on day 1 is  -3.2199999999999998
prosp on day 1 is  -1.22
prosp on day 1 is  0.78
counts from the previous value of 1  is  6 cutoff price is now 7 for day  1
-2.1855
prosp on day 2 is  -0.7354999999999998
prosp on day 2 is  0.7145
counts from the previous value of 7  is  2 cutoff price is now 9 for day  2
-0.38956
prosp on day 3 is  0.8904399999999999
counts from the previous value of 9  is  1 cutoff price is now 10 for day  3
0.22464759999999984
counts from the previous value of 10  is  0 cutoff price is now 10 for day  4
-0.24602839600000015
prosp on day 5 is  0.9639716039999999
counts from the previous value of 10  is  1 cutoff price is now 11 for day  5
0.5383758634
counts from the previous value of 11  is  0 cutoff price is now 11 for day  6
0.22761948388999997
counts from the previous value of 11  is  0 cutoff price is now 11 for day  7
-0.036523438693500054
prosp on 

In [ ]:
losstest

0.18181818181818177

In [ ]:
GUESS=[.5,.5,1,.5,30]

# Optimization Functions


In [ ]:
def dict_constraint(xk):
  return xk[1]-xk[0]

In [ ]:
check=lambda params: params if params[0]<=params[1] else "Alpha greater than Beta"

In [ ]:
from scipy.optimize import differential_evolution
from scipy.optimize import LinearConstraint
def differential_evolution_participant(participant,save):
    part=participant
    g=np.array([.5,.5,1,.5,60])
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(7,68)] # changing the bounds might be the reason why it's messing up, but a and b cannot be greater than 1 keep in mind
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    result = differential_evolution(func=fit_errorPT_TW,x0=g,bounds=bounds, args=(part,), strategy="best1bin", maxiter=30000, tol=0.0002, disp=True,updating="immediate",constraints=de_lc,polish=True,integrality=[False,False,False,False,True] ) # can change back to get_loss_one or avg error if needed
    best_fit = result.x
    print(result.message)
    if save:
      save_fit(best_fit,participant+1,"diffev")
    return best_fit


In [ ]:
from scipy.optimize import direct
def direct_participant(participant,dum):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    result = direct(func=fit_errorPT_TW, bounds=bounds, args=(part,), maxiter=10000, locally_biased=False)

    best_fit = result.x
    print (result.message)
    if dum:
      save_fit(check(best_fit),participant+1,'direct')
    return best_fit

In [ ]:
from scipy.optimize import shgo
def shgo_participant(participant):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    dc=({'type': 'ineq','fun':dict_constraint})
    result = shgo(func=fit_errorPT_TW, bounds=bounds, args=(part,) , options={'maxiter':10000},constraints=dc)

    best_fit = result.x
    print (result.message)

    return best_fit

In [ ]:
from scipy.optimize import minimize
def nelder_mead_participant(participant, guess):
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), bounds=bounds, method='Nelder-Mead', options={"fatol":.15})
  return result.x

cob_eq=lambda x: x[1]-x[0]

def cobyla_participant(participant,guess):
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), method='COBYLA', bounds=bounds, constraints={"type":"ineq", "fun":cob_eq}, options={"tol":.13})
  return result.x

In [ ]:
def global_polish(participant,starter,count,glazer):
  silly=None
  s=''
  if starter=='diffev':
    silly=differential_evolution_participant
    s='neighbors'
  elif starter=='direct':
    silly= direct_participant
    s='polished_direct'


  ret='didnt work'
  start=silly(participant,False)
  p=None
  if count==1:
    if glazer=='NM':
      p=nelder_mead_participant
    elif glazer=='CB':
      p=cobyla_participant
    end=p(participant, start)
    save_fit(end,participant+1,s)
    ret=end


    ret =check(end)
  elif count==2:
    end1=nelder_mead_participant(participant,start)
    end2=cobyla_participant(participant,start)
    dickle={
        "NM": check(end1),
        "CB":(end2)
    }
    save_fit(dickle,participant+1,s)
    ret=dickle
  return ret





In [ ]:
%pip install Py-BOBYQA

  Preparing metadata (setup.py) ... done
  Created wheel for Py-BOBYQA: filename=Py_BOBYQA-1.4-py3-none-any.whl size=43290 sha256=9ba6533790cb4501cae19849c21aca500ad9ce93c410693467fea52b5891d60b
  Stored in directory: /root/.cache/pip/wheels/15/95/87/07ebd66ef9a50c600e4f8c7fc8e8932ba2f17bba71255ac3bb
Successfully built Py-BOBYQA


In [ ]:
import pybobyqa as bob

In [ ]:
def bobyqa_participant(participant):

  g=np.array([.5,.5,1,.5,60])
  #bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]
  b=(np.array([0,0,1,0,2]),np.array([1,1,2,1,68]))
  error_fn=fit_errorPT_TW
  result=bob.solve(error_fn,g,args=(participant,) ,bounds=b,seek_global_minimum=True,print_progress=True,maxfun=5000, rhobeg=.4)
  answer=0

  #print("Success!")
  print(result.msg, result.nf)
  print(result)
  answer=result.x
  save_fit(answer,participant+1,'bobby')

  return answer


In [ ]:
%pip install nlopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.7/423.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
import nlopt
def nlopt_participant(participant):
  #algorithm=nlopt.NLOPT_GN_CRS2_LM # NLOPT_GN_CRS2_LM CRS technique...
  algorithm="NLOPT_GN_CRS2_LM"
  opt = nlopt.opt(algorithm, 5)
  func=lambda params: fit_errorPT_TW(params,participant)
  opt.set_min_objective(func)
  lower_bounds=[0,0,1,0,2]
  upper_bounds=[1,1,2,1,68]
  opt.set_lower_bounds(lower_bounds)
  opt.set_upper_bounds(upper_bounds)
  stopval=0.15
  opt.set_stopval(stopval)
  maxtime=25200
  # figure out constraints
  opt.set_maxtime(maxtime)

  opt.optimize(GUESS)
  opt_val = opt.last_optimum_value()
  result = opt.last_optimize_result()
  return result


In [ ]:
%pip install dlib

In [ ]:
import dlib
def dlib_participant(participant):
  func=lambda a,b,l,g,tw: fit_errorPT_TW(np.array([a,b,l,g,tw]),participant)
  lower_bounds=[0,0,1,0,2]
  upper_bounds=[1,1,2,1,68]
  x,y = dlib.find_min_global(func,
                           lower_bounds,  # Lower bound constraints on x0 and x1 respectively
                           upper_bounds,    # Upper bound constraints on x0 and x1 respectively
                           50000)         # The number of times find_min_global() will call func
  return x

In [ ]:
!pip install nevergrad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.7/477.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 24.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
# prompt: Define a python function that will return the optimized result of fit_error_PTTW for all 5 parameters using nevergrad
import nevergrad as ng
def nevergrad_participant(participant):
  fit_func=lambda a,b,l,g,tw: modified_func(a,b,l,g,tw,participant)

  arg_a=ng.p.Scalar().set_bounds(0,1)
  arg_b=ng.p.Scalar().set_bounds(0,1)
  arg_l=ng.p.Scalar().set_bounds(0,2)
  arg_g=ng.p.Scalar().set_bounds(0,1)
  arg_tw=ng.p.Scalar().set_bounds(0,68).set_integer_casting()

  params=ng.p.Instrumentation(arg_a,arg_b,arg_l, arg_g,arg_tw)

  optimizer = ng.optimizers.NGOpt(params,budget=30000)

  result = optimizer.minimize(fit_func)
  save_fit(result.value,participant+1,"brute")
  return result.value


In [ ]:
PARAM_LIST=["a","b","l","g","tw"]

In [ ]:
MAX_ITER=7000

In [ ]:
from itertools import product
def exhaustive_fit_with_guess_one_subject(participant,precision,guess,target):
                                #subject:        int,

        """Performs the exhaustive fit algorithm for one subject and saves the best fit.
        The exhaustive fit algorithm consists of iterating through all possible parameter values (with a given level of precision) and accepting the best.
        Inputs:
            subject: the participant's number in the dataframe.
            precision: the amount to increment each value when iterating through all possible values.
            verbose: set to True to get progress bars for the fitting.
            error_type: should the error be calculated as the absolute difference
                        between the prediction and the amount, or as
                        the difference in proportion of goods sold. report.docx
                        seems to use proportional.
            tw: if provided, tw will stay at this constant level."""

        #prev_best_fit: Parameters = self.best_fits[subject]
        #initial_pred = self.predict_one_subject(subject=subject, fit=prev_best_fit)
        #lowest_error = self.mean_error_one_subject_proportion(subject, initial_pred)
        prev_best_fit=guess #array/list
        best_fit = prev_best_fit #array/ list
        initial_pred=fitPT_TW(participant,guess[0],guess[1],guess[2],guess[3],guess[4])[0] #a,b,l,g,tw

        # Set lowest error to initial error

       #lowest_error=fit_errorPT_TW(prev_best_fit,participant)  # d2a error
        lowest_error=count_errorPT_TW(prev_best_fit,participant)
        # Get lists of all possible values for all free params
        low  = lambda fit, old_precision, floor: max(floor, fit - old_precision/2) if fit else 0  #basically setting the floor which is 0.01,
        high = lambda fit, old_precision, ceil:  min(ceil,  fit + old_precision/2) if fit else 0  # setting the ceil which is 1 for except lam which it is 2
        valid_parameter_ranges= {                        # Dict[str, List[float]] = {

            "a": list(np.arange(
                            .01,#low(prev_best_fit[0],  prev_precision, precision), # here he is setting the minimum value to the value of the guess or the floor value which
                             1, #high(prev_best_fit[0], prev_precision, 1),# + EPS,
                            precision
                )),
            "b": list(np.arange(
                            .01,#low(prev_best_fit[1],  prev_precision, precision),
                            1,  #high(prev_best_fit[1], prev_precision, 1),# + EPS,
                            precision
                )),
            "l": list(np.arange(
                            1,#low(prev_best_fit[2],  prev_precision, 1),
                            2,#high(prev_best_fit[2], prev_precision, 2),# + EPS,
                            precision
                )),

            "g": list(np.arange(
                            .01,#low(prev_best_fit[3],  prev_precision, precision),
                             1,#high(prev_best_fit[3], prev_precision, 1),# + EPS,
                            precision
                )),

            "tw": list(np.arange(2, 68, 1))
        }
        #if tw is not None:
         # valid_parameter_ranges["tw"] = [tw]

        # Remove data on non-free params:
      #  ranges: List[List[Any]] = [valid_parameter_ranges[param] if param in self.free_params
         #                                                        else [None]
         #                                                        for param in PARAM_LIST ]
        ranges=[valid_parameter_ranges[param] for param in PARAM_LIST]
        # Get all possible values via cartesian product
        all_possible_fits = list(product(*ranges)) # takes every possible range

        # Iterate through every possible value
        iterations = 1
        for range in ranges:
            iterations *= len(range)
       # for fit in tqdm(all_possible_fits,

                      #  leave=False,
                       # total=iterations,
                       # desc="Attempting all fits..."):
        count=0
        for fit in all_possible_fits:
            count+=1
            print("iter:",count)
            if (lowest_error<=target) or (count>MAX_ITER):
              break
            # Skip fits where a > b
            if fit[0] > fit[1]:
                continue
            # Keep fits where xl-.1 <= xg <= xl
         #   if (fit[2] is not None and fit[3] is not None) and (fit[2] > fit[3] or fit[2] < fit[3]-0.1):
               # continue

            # Predict sale amounts based on fit

           # predictions: List[int] = self.predict_one_subject(subject, fit_params)
            fit_params=fit

            preds=fitPT_TW(participant,fit_params[0],fit_params[1],fit_params[2],fit_params[3],fit_params[4])
            # Get error for the given fit
            #error: float = error_fn(subject, predictions)
            #error=fit_errorPT_TW(np.array(fit_params),participant)
            error=count_errorPT_TW(np.array(fit_params),participant)
            # Check if it's the best so far:
            if error < lowest_error:
                lowest_error = error
                best_fit  = fit_params

            elif error == lowest_error:
                if fit_params[0] is not None and fit_params[0] >= best_fit[0]:
                    best_fit = fit_params
                elif not fit_params[0]:
                    best_fit = fit_params
                #self.all_best_fits[subject].append(fit_params)

        save_fit(best_fit,participant+1,"brute")
        return best_fit



In [ ]:
def takeSecond(elem):
  return elem[1]

In [ ]:
def sorted_ranking(lst):
  entire_tuple_list = []
  for i in range (len(lst)):
    tempTup = (i,count_error(lst[i],i))
    entire_tuple_list.append(tempTup)
  entire_tuple_list.sort(key=takeSecond)
  return entire_tuple_list

In [ ]:
# import os
# fit_data=os.listdir('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits')
# fit_data[1][12:14]
# fit_data
# diffev_fits=[(int(path[12:14]),load_fit(DIFEV_SAVE_PATH+path)) for path in fit_data]
# df_days
# diffev_fits
# pt_tw_errors=[(i-1,count_errorPT_TW(fit,i-1)) for (i,fit) in diffev_fits]
# pt_tw_errors

In [ ]:
# def sort_tuples(lst):
#     return sorted(lst, key=lambda x: x[0])


In [ ]:
# sorted_errors=sort_tuples(pt_tw_errors)

In [ ]:
# len(sorted_errors)

57

In [ ]:
# sorted_errors[33]

In [ ]:
# sorted_errors

[(0, 0),
 (1, 5),
 (2, 5),
 (3, 9),
 (4, 4),
 (5, 0),
 (6, 11),
 (7, 0),
 (8, 27),
 (9, 0),
 (10, 5),
 (11, 10),
 (12, 11),
 (13, 11),
 (14, 11),
 (15, 26),
 (16, 12),
 (17, 22),
 (18, 19),
 (19, 18),
 (20, 8),
 (21, 16),
 (22, 55),
 (23, 31),
 (24, 20),
 (25, 39),
 (26, 30),
 (27, 22),
 (28, 23),
 (29, 31),
 (30, 49),
 (31, 29),
 (32, 58),
 (33, 43),
 (34, 79),
 (35, 27),
 (36, 61),
 (37, 35),
 (38, 35),
 (39, 74),
 (40, 69),
 (41, 49),
 (42, 42),
 (43, 37),
 (44, 35),
 (45, 55),
 (46, 54),
 (47, 48),
 (48, 55),
 (49, 53),
 (50, 50),
 (51, 58),
 (52, 72),
 (53, 79),
 (54, 68),
 (55, 68),
 (56, 69)]

In [ ]:
# sorted_errors.remove(sorted_errors[24])
# sorted_errors.remove(sorted_errors[33])
# sorted_errors.remove(sorted_errors[6])


In [ ]:
# our_errors=pd.read_csv('data/our_errors - fixed our_errors.csv')

In [ ]:
# our_errors

,Unnamed: 0,our_error
0,0,0
1,1,1
2,2,3
3,3,9
4,4,4
5,5,0
6,6,9
7,7,0
8,8,28
9,9,0


In [ ]:
# our_errors=our_errors.drop(["Unnamed: 0"], axis=1)

In [ ]:
# our_errors

,our_error
0,0
1,1
2,3
3,9
4,4
5,0
6,9
7,0
8,28
9,0


In [ ]:
# z=zip(list(our_errors.index),list(our_errors['our_error'].values))

In [ ]:
# PT_errs=list(z)

In [ ]:
# PT_errs

[(0, 0),
 (1, 1),
 (2, 3),
 (3, 9),
 (4, 4),
 (5, 0),
 (6, 9),
 (7, 0),
 (8, 28),
 (9, 0),
 (10, 5),
 (11, 10),
 (12, 11),
 (13, 11),
 (14, 11),
 (15, 26),
 (16, 17),
 (17, 22),
 (18, 20),
 (19, 21),
 (20, 10),
 (21, 9),
 (22, 44),
 (23, 31),
 (24, 20),
 (25, 39),
 (26, 25),
 (27, 9),
 (28, 16),
 (29, 25),
 (30, 49),
 (31, 29),
 (32, 41),
 (33, 43),
 (34, 73),
 (35, 27),
 (36, 66),
 (37, 35),
 (38, 34),
 (39, 71),
 (40, 72),
 (41, 53),
 (42, 42),
 (43, 36),
 (44, 35),
 (45, 55),
 (46, 54),
 (47, 48),
 (48, 55),
 (49, 53),
 (50, 64),
 (51, 57),
 (52, 74),
 (53, 79),
 (54, 68),
 (55, 69),
 (56, 69)]

In [ ]:
# sorted_errors

[(0, 0),
 (1, 5),
 (2, 5),
 (3, 9),
 (4, 4),
 (5, 0),
 (6, 11),
 (7, 0),
 (8, 27),
 (9, 0),
 (10, 5),
 (11, 10),
 (12, 11),
 (13, 11),
 (14, 11),
 (15, 26),
 (16, 12),
 (17, 22),
 (18, 19),
 (19, 18),
 (20, 8),
 (21, 16),
 (22, 55),
 (23, 31),
 (24, 20),
 (25, 39),
 (26, 30),
 (27, 22),
 (28, 23),
 (29, 31),
 (30, 49),
 (31, 29),
 (32, 58),
 (33, 43),
 (34, 79),
 (35, 27),
 (36, 61),
 (37, 35),
 (38, 35),
 (39, 74),
 (40, 69),
 (41, 49),
 (42, 42),
 (43, 37),
 (44, 35),
 (45, 55),
 (46, 54),
 (47, 48),
 (48, 55),
 (49, 53),
 (50, 50),
 (51, 58),
 (52, 72),
 (53, 79),
 (54, 68),
 (55, 68),
 (56, 69)]

In [ ]:
# PT_better=[]
# PT_TW_better=[]
# equals=[]
# for i in range(57):
#   Pt_tw=sorted_errors[i]
#   Pt=PT_errs[i]
#   if Pt_tw[1]<Pt[1]:
#     PT_TW_better.append(i)
#   elif Pt_tw[1]==Pt[1]:
#     equals.append(i)
#   else:
#     PT_better.append(i)


In [ ]:
# PT_better

[1, 2, 6, 21, 22, 26, 27, 28, 29, 32, 34, 38, 39, 43, 51]

ALL OF THE ABOVE MUST BE CHANGED

improved pt_tw

participant 2 (i=1) fit=
array([ 0.40763567, 0.43357354, 1.03015975, 0.96945102, 14.00001488])
count error=1


In [ ]:
# loaded_bf23_dif= load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/participant 23 January 02, 2024.pickle')
# loaded_bf23_dir=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/participant 23 January 02, 2024.pickle')


In [ ]:
# loaded_bf23_dif

{'NM': array([ 0.58557851,  0.8914878 ,  1.01868328,  0.82599894, 63.        ]),
 'CB': array([ 0.49489318,  0.86503313,  1.        ,  0.73866478, 63.        ])}

In [ ]:
# count_errorPT_TW(loaded_bf23_dir['NM'],22)

-15.28376491449813
prosp on day 1 is  -12.91646472415326
prosp on day 1 is  -10.506615497686429
prosp on day 1 is  -8.10427789219551
prosp on day 1 is  -5.76203699201713
prosp on day 1 is  -3.546488669862444
prosp on day 1 is  -1.5055814705312427
prosp on day 1 is  0.3702502452094359
counts from the previous value of 1  is  7 cutoff price is now 8 for day  1
-3.1004009613139973
prosp on day 2 is  -1.0619229823561016
prosp on day 2 is  0.8186341372606094
counts from the previous value of 8  is  2 cutoff price is now 10 for day  2
0.026137806191481427
counts from the previous value of 10  is  0 cutoff price is now 10 for day  3
-0.5753705553391701
prosp on day 4 is  2.6085602400231465
counts from the previous value of 10  is  1 cutoff price is now 11 for day  4
3.0615549840379157
counts from the previous value of 11  is  0 cutoff price is now 11 for day  5
3.8279680513908456
counts from the previous value of 11  is  0 cutoff price is now 11 for day  6
4.86842243622095
counts from the pre

44

In [ ]:
# loaded_bf23_dir

{'NM': array([ 0.5       ,  0.83333333,  1.01851852,  0.75925926, 57.        ]),
 'CB': array([ 0.5       ,  0.83333333,  1.01851852,  0.75925926, 57.        ])}

In [ ]:
# lowest_error=100
# best_tw=0
# for tw in range(2,68):
#   err=count_errorPT_TW([1.0 , 1, 1,.355,tw],21)
#   if err<lowest_error:
#     best_tw=tw
#     lowest_error=err


Streaming output truncated to the last 5000 lines.
util PT :  5030807346.143688
util PT :  6288509182.679611
util PT :  7546211019.215535
util PT :  8803912855.751455
util PT :  10061614692.287376
cutoff value of 9 bigger than price 3 HOLD PREDICTED 
cutoff value of 9 bigger than price 8 HOLD PREDICTED 
cutoff value of 9 bigger than price 3 HOLD PREDICTED 
cutoff value of 9 bigger than price 4 HOLD PREDICTED 
cutoff value of 9 bigger than price 1 HOLD PREDICTED 
cutoff value of 9 bigger than price 7 HOLD PREDICTED 
cutoff value of 9 bigger than price 8 HOLD PREDICTED 
cutoff value of 9 bigger than price 4 HOLD PREDICTED 
cutoff value of 9 bigger than price 5 HOLD PREDICTED 
cutoff value of 9 bigger than price 7 HOLD PREDICTED 
cutoff value of 9 bigger than price 5 HOLD PREDICTED 
cutoff value of 9 bigger than price 3 HOLD PREDICTED 
util PT :  66357.7695954165
util PT :  132715.539190833
util PT :  199073.3087862495
util PT :  265431.078381666
util PT :  331788.8479770824
util PT :  39

In [ ]:
# lowest_error

8

In [ ]:
# best_tw

61

In [ ]:
# count_errorPT_TW([1,1,1,.35,61],21)

-47.69184224881904
prosp on day 1 is  -40.566363967927145
prosp on day 1 is  -33.44088568703524
prosp on day 1 is  -26.315407406143343
prosp on day 1 is  -19.189929125251446
prosp on day 1 is  -12.06445084435954
prosp on day 1 is  -4.93897256346764
prosp on day 1 is  2.186505717424262
counts from the previous value of 1  is  7 cutoff price is now 8 for day  1
-9.701339576264509
prosp on day 2 is  0.2125926818189683
counts from the previous value of 8  is  1 cutoff price is now 9 for day  2
0.07331150981304191
counts from the previous value of 9  is  0 cutoff price is now 9 for day  3
0.3002986425584382
counts from the previous value of 9  is  0 cutoff price is now 9 for day  4
0.7622519344755148
counts from the previous value of 9  is  0 cutoff price is now 9 for day  5
1.7023970490110685
counts from the previous value of 9  is  0 cutoff price is now 9 for day  6
3.6157351545088545
counts from the previous value of 9  is  0 cutoff price is now 9 for day  7
7.509668813631606
counts from

8

In [ ]:
# PT_TW_better

[8, 16, 18, 19, 20, 36, 40, 41, 50, 52, 55]

In [ ]:
# equals

[0,
 3,
 4,
 5,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 23,
 24,
 25,
 30,
 31,
 33,
 35,
 37,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 53,
 54,
 56]